In [1]:
import math
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime, date

In [2]:
print(tf.__version__)

1.8.0


In [3]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook

import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
df = pd.read_pickle("indego_df_model.pkl")

# Train/Valid/Test dataset split

In [6]:
df.tail(3); df.shape

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,...,plan_duration,trip_route_category,passholder_type,duration_type,distance,start_weekday,start_hour,start_date,isBday,start_month
1955376,4150099,7.0,2015-06-30 23:41:00,2015-06-30 23:48:00,3023,39.95048,-75.17286,3045,39.94792,-75.16237,...,30,One Way,Indego30,very short,0.938421,Tuesday,23,2015-06-30,1.0,6
1955377,4150100,8.0,2015-06-30 23:48:00,2015-06-30 23:56:00,3026,39.94138,-75.14564,3045,39.94792,-75.16237,...,30,One Way,Indego30,very short,1.600920,Tuesday,23,2015-06-30,1.0,6
1955378,4150101,4.0,2015-06-30 23:49:00,2015-06-30 23:53:00,3063,39.94633,-75.16980,3063,39.94633,-75.16980,...,0,Round Trip,Walk-up,very short,0.000000,Tuesday,23,2015-06-30,1.0,6


(1955379, 21)

In [7]:
df.reset_index().groupby('passholder_type').apply(lambda x: x.sample(frac=0.7, random_state=123)).reset_index(drop=True).head(3)

,index,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,...,plan_duration,trip_route_category,passholder_type,duration_type,distance,start_weekday,start_hour,start_date,isBday,start_month
0,1360232,115216262,13.0,2017-04-23 21:18:00,2017-04-23 21:31:00,3027,39.957081,-75.201767,3117,39.978409,...,30,One Way,Indego30,very short,3.035017,Sunday,21,2017-04-23,0.0,4
1,1199322,4685611,7.0,2016-05-11 18:16:00,2016-05-11 18:23:00,3022,39.954720,-75.183230,3027,39.957080,...,30,One Way,Indego30,very short,1.601902,Wednesday,18,2016-05-11,1.0,5
2,1895998,4080293,23.0,2015-05-27 17:32:00,2015-05-27 17:55:00,3009,39.955760,-75.189820,3063,39.946330,...,30,One Way,Indego30,short,2.002933,Wednesday,17,2015-05-27,1.0,5


In [8]:
df_train_valid = df.reset_index().groupby('passholder_type').apply(lambda x: x.sample(frac=0.9, random_state=123)).reset_index(drop=True).set_index('index')
df_test = df.drop(df_train_valid.index)

In [9]:
df_train_valid.shape; df_test.shape; df_test.shape[0]/df_train_valid.shape[0]

(1759842, 21)

(195537, 21)

0.11111054287828112

In [10]:
df_train = df_train_valid.reset_index().groupby('passholder_type').apply(lambda x: x.sample(frac=0.8, random_state=1234)).reset_index(drop=True).set_index('index')
df_valid = df_train_valid.drop(df_train.index)

In [11]:
df_train.shape; df_valid.shape; df_test.shape; df_valid.shape[0]/df_train.shape[0]

(1407874, 21)

(351968, 21)

(195537, 21)

0.24999964485458215

# 2. Benchmark

In [12]:
def compute_rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted)**2))

def print_rmse(df, name):
    print("{1} RMSE = {0}".format(compute_rmse(df['duration'], df_train["duration"].mean()), name))

In [13]:
print_rmse(df_train, 'Train')
print_rmse(df_valid, 'Valid') 
print_rmse(df_test, 'Test')

Train RMSE = 54.914272416647556
Valid RMSE = 53.555993575575
Test RMSE = 53.25920503905649


In [14]:
col_list = ["start_station", "passholder_type", "start_weekday", "start_hour"]

In [15]:
# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = df[col_list],
        y = df["duration"],
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000,
        num_threads = 1
    )

In [16]:
def make_prediction_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = df[col_list],
        y = None,
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000,
        num_threads = 1
  )

In [17]:
df_train["start_station"].nunique(); df_train["bike_id"].nunique()

127

1996

In [18]:
# Define feature columns
def create_feature_cols():
    return [
        tf.feature_column.categorical_column_with_vocabulary_list(
            key='passholder_type',
            vocabulary_list=df["passholder_type"].unique().tolist()),
#         tf.feature_column.categorical_column_with_identity(
#             key='start_station',
#             num_buckets=127),
        tf.feature_column.categorical_column_with_hash_bucket (
            key='start_station',
            hash_bucket_size=130,
            dtype=tf.int64),
        tf.feature_column.crossed_column(["start_weekday", "start_hour"], 168)
    ]

# 4. Linear Regression

In [19]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.LinearRegressor(model_dir = output_dir, feature_columns = create_feature_cols())
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(df_train, 8), 
                                        max_steps = num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(df_valid, 1), 
                                      steps = None,
                                      start_delay_secs = 1, # start evaluating after N seconds, 
                                      throttle_secs = 10)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [107]:
# Run the model
OUTDIR = './trained_model'
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 500)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './trained_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x16b7ce4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow

In [108]:
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = create_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_input_fn(df_train, num_epochs = 1))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './trained_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x135662ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 53099.0, s

INFO:tensorflow:global_step/sec: 72.0545
INFO:tensorflow:loss = 15354.6, step = 7501 (1.390 sec)
INFO:tensorflow:global_step/sec: 60.0523
INFO:tensorflow:loss = 771171.0, step = 7601 (1.679 sec)
INFO:tensorflow:global_step/sec: 49.3443
INFO:tensorflow:loss = 15944.8, step = 7701 (2.011 sec)
INFO:tensorflow:global_step/sec: 68.0616
INFO:tensorflow:loss = 136046.0, step = 7801 (1.475 sec)
INFO:tensorflow:global_step/sec: 64.8291
INFO:tensorflow:loss = 664701.0, step = 7901 (1.535 sec)
INFO:tensorflow:global_step/sec: 74.9448
INFO:tensorflow:loss = 32797.1, step = 8001 (1.330 sec)
INFO:tensorflow:global_step/sec: 102.039
INFO:tensorflow:loss = 11018.3, step = 8101 (0.981 sec)
INFO:tensorflow:global_step/sec: 85.0401
INFO:tensorflow:loss = 47598.1, step = 8201 (1.184 sec)
INFO:tensorflow:global_step/sec: 91.4644
INFO:tensorflow:loss = 17553.2, step = 8301 (1.087 sec)
INFO:tensorflow:global_step/sec: 93.5022
INFO:tensorflow:loss = 47559.9, step = 8401 (1.068 sec)
INFO:tensorflow:global_step

In [109]:
def print_rmse(model, name, df):
    metrics = model.evaluate(input_fn = make_input_fn(df, 1))
    print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-31-12:59:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-31-12:59:58
INFO:tensorflow:Saving dict for global step 11000: average_loss = 2774.64, global_step = 11000, loss = 355122.0
RMSE on validation dataset = 52.67486572265625


# 5. Neural Network

In [78]:
feature_col1 = tf.feature_column.categorical_column_with_vocabulary_list(
            key='passholder_type',
            vocabulary_list=df["passholder_type"].unique().tolist())

feature_col2 = tf.feature_column.categorical_column_with_hash_bucket (
            key='start_station',
            hash_bucket_size=130,
            dtype=tf.int64)

feature_col3 = tf.feature_column.crossed_column(["start_weekday", "start_hour"], 168)

my_feature_columns = [
    tf.feature_column.indicator_column(feature_col1),
    tf.feature_column.indicator_column(feature_col2),
    tf.feature_column.indicator_column(feature_col3)
]


In [79]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = './trained_DNNmodel'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = my_feature_columns, model_dir = OUTDIR)
model.train(input_fn = make_input_fn(df_train, num_epochs = 1));
print_rmse(model, 'validation', df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './trained_DNNmodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x14aaf85f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./trained_DNNmodel/model.ckpt.
INFO:tensorflow:loss = 2327

INFO:tensorflow:global_step/sec: 175.208
INFO:tensorflow:loss = 10705.9, step = 7501 (0.570 sec)
INFO:tensorflow:global_step/sec: 190.478
INFO:tensorflow:loss = 18690.6, step = 7601 (0.525 sec)
INFO:tensorflow:global_step/sec: 184.133
INFO:tensorflow:loss = 10089.8, step = 7701 (0.543 sec)
INFO:tensorflow:global_step/sec: 192.018
INFO:tensorflow:loss = 22665.1, step = 7801 (0.521 sec)
INFO:tensorflow:global_step/sec: 177.324
INFO:tensorflow:loss = 7937.2, step = 7901 (0.564 sec)
INFO:tensorflow:global_step/sec: 170.778
INFO:tensorflow:loss = 32437.9, step = 8001 (0.586 sec)
INFO:tensorflow:global_step/sec: 184.171
INFO:tensorflow:loss = 15826.6, step = 8101 (0.543 sec)
INFO:tensorflow:global_step/sec: 189.121
INFO:tensorflow:loss = 41551.1, step = 8201 (0.529 sec)
INFO:tensorflow:global_step/sec: 182.458
INFO:tensorflow:loss = 23535.5, step = 8301 (0.548 sec)
INFO:tensorflow:global_step/sec: 187.378
INFO:tensorflow:loss = 117033.0, step = 8401 (0.534 sec)
INFO:tensorflow:global_step/se

INFO:tensorflow:loss = 59441.8, step = 15801 (0.595 sec)
INFO:tensorflow:global_step/sec: 179.128
INFO:tensorflow:loss = 707050.0, step = 15901 (0.565 sec)
INFO:tensorflow:global_step/sec: 114.404
INFO:tensorflow:loss = 27958.9, step = 16001 (0.873 sec)
INFO:tensorflow:global_step/sec: 94.696
INFO:tensorflow:loss = 154370.0, step = 16101 (1.062 sec)
INFO:tensorflow:global_step/sec: 127.674
INFO:tensorflow:loss = 11347.6, step = 16201 (0.781 sec)
INFO:tensorflow:global_step/sec: 109.859
INFO:tensorflow:loss = 24378.9, step = 16301 (0.910 sec)
INFO:tensorflow:global_step/sec: 110.351
INFO:tensorflow:loss = 21121.1, step = 16401 (0.901 sec)
INFO:tensorflow:global_step/sec: 105.195
INFO:tensorflow:loss = 14444.8, step = 16501 (0.950 sec)
INFO:tensorflow:global_step/sec: 101.996
INFO:tensorflow:loss = 12383.0, step = 16601 (0.978 sec)
INFO:tensorflow:global_step/sec: 107.838
INFO:tensorflow:loss = 45932.8, step = 16701 (0.931 sec)
INFO:tensorflow:global_step/sec: 119.239
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 109.764
INFO:tensorflow:loss = 15146.7, step = 24201 (0.904 sec)
INFO:tensorflow:global_step/sec: 116.751
INFO:tensorflow:loss = 96814.3, step = 24301 (0.858 sec)
INFO:tensorflow:global_step/sec: 153.917
INFO:tensorflow:loss = 25964.5, step = 24401 (0.647 sec)
INFO:tensorflow:global_step/sec: 175.842
INFO:tensorflow:loss = 79040.1, step = 24501 (0.573 sec)
INFO:tensorflow:global_step/sec: 152.444
INFO:tensorflow:loss = 309116.0, step = 24601 (0.650 sec)
INFO:tensorflow:global_step/sec: 161.031
INFO:tensorflow:loss = 1.05876e+06, step = 24701 (0.620 sec)
INFO:tensorflow:global_step/sec: 124.752
INFO:tensorflow:loss = 643237.0, step = 24801 (0.809 sec)
INFO:tensorflow:global_step/sec: 118.74
INFO:tensorflow:loss = 20322.5, step = 24901 (0.843 sec)
INFO:tensorflow:global_step/sec: 126.589
INFO:tensorflow:loss = 176513.0, step = 25001 (0.785 sec)
INFO:tensorflow:global_step/sec: 112.304
INFO:tensorflow:loss = 59997.1, step = 25101 (0.894 sec)
INFO:tensorflo

INFO:tensorflow:loss = 169762.0, step = 32501 (0.696 sec)
INFO:tensorflow:global_step/sec: 111.045
INFO:tensorflow:loss = 14567.3, step = 32601 (0.903 sec)
INFO:tensorflow:global_step/sec: 123.926
INFO:tensorflow:loss = 21793.3, step = 32701 (0.808 sec)
INFO:tensorflow:global_step/sec: 103.727
INFO:tensorflow:loss = 46500.8, step = 32801 (0.960 sec)
INFO:tensorflow:global_step/sec: 103.047
INFO:tensorflow:loss = 25277.6, step = 32901 (0.973 sec)
INFO:tensorflow:global_step/sec: 114.197
INFO:tensorflow:loss = 20422.5, step = 33001 (0.879 sec)
INFO:tensorflow:global_step/sec: 113.374
INFO:tensorflow:loss = 23396.3, step = 33101 (0.880 sec)
INFO:tensorflow:global_step/sec: 134.319
INFO:tensorflow:loss = 16682.0, step = 33201 (0.746 sec)
INFO:tensorflow:global_step/sec: 126.829
INFO:tensorflow:loss = 23883.1, step = 33301 (0.788 sec)
INFO:tensorflow:global_step/sec: 123.719
INFO:tensorflow:loss = 30268.9, step = 33401 (0.810 sec)
INFO:tensorflow:global_step/sec: 116.746
INFO:tensorflow:los

...

INFO:tensorflow:global_step/sec: 161.773
INFO:tensorflow:loss = 20489.3, step = 82601 (0.613 sec)
INFO:tensorflow:global_step/sec: 189.586
INFO:tensorflow:loss = 102701.0, step = 82701 (0.531 sec)
INFO:tensorflow:global_step/sec: 176.598
INFO:tensorflow:loss = 10937.9, step = 82801 (0.563 sec)
INFO:tensorflow:global_step/sec: 174.908
INFO:tensorflow:loss = 20082.4, step = 82901 (0.571 sec)
INFO:tensorflow:global_step/sec: 157.754
INFO:tensorflow:loss = 14089.3, step = 83001 (0.634 sec)
INFO:tensorflow:global_step/sec: 170.378
INFO:tensorflow:loss = 15918.2, step = 83101 (0.587 sec)
INFO:tensorflow:global_step/sec: 163.18
INFO:tensorflow:loss = 23078.1, step = 83201 (0.613 sec)
INFO:tensorflow:global_step/sec: 128.878
INFO:tensorflow:loss = 249344.0, step = 83301 (0.777 sec)
INFO:tensorflow:global_step/sec: 136.387
INFO:tensorflow:loss = 24437.9, step = 83401 (0.737 sec)
INFO:tensorflow:global_step/sec: 128.729
INFO:tensorflow:loss = 56718.6, step = 83501 (0.774 sec)
INFO:tensorflow:glo

INFO:tensorflow:loss = 16755.2, step = 90901 (0.919 sec)
INFO:tensorflow:global_step/sec: 107.953
INFO:tensorflow:loss = 38391.2, step = 91001 (0.925 sec)
INFO:tensorflow:global_step/sec: 127.054
INFO:tensorflow:loss = 26368.5, step = 91101 (0.792 sec)
INFO:tensorflow:global_step/sec: 132.152
INFO:tensorflow:loss = 15654.2, step = 91201 (0.752 sec)
INFO:tensorflow:global_step/sec: 128.325
INFO:tensorflow:loss = 18130.1, step = 91301 (0.778 sec)
INFO:tensorflow:global_step/sec: 127.218
INFO:tensorflow:loss = 23918.5, step = 91401 (0.787 sec)
INFO:tensorflow:global_step/sec: 141.244
INFO:tensorflow:loss = 16707.6, step = 91501 (0.709 sec)
INFO:tensorflow:global_step/sec: 104.347
INFO:tensorflow:loss = 20740.1, step = 91601 (0.963 sec)
INFO:tensorflow:global_step/sec: 100.487
INFO:tensorflow:loss = 381125.0, step = 91701 (0.995 sec)
INFO:tensorflow:global_step/sec: 121.247
INFO:tensorflow:loss = 177944.0, step = 91801 (0.820 sec)
INFO:tensorflow:global_step/sec: 153.3
INFO:tensorflow:loss

INFO:tensorflow:global_step/sec: 123.154
INFO:tensorflow:loss = 14194.0, step = 99301 (0.810 sec)
INFO:tensorflow:global_step/sec: 109.55
INFO:tensorflow:loss = 62701.8, step = 99401 (0.915 sec)
INFO:tensorflow:global_step/sec: 98.6241
INFO:tensorflow:loss = 178720.0, step = 99501 (1.016 sec)
INFO:tensorflow:global_step/sec: 113.524
INFO:tensorflow:loss = 37304.6, step = 99601 (0.880 sec)
INFO:tensorflow:global_step/sec: 115.075
INFO:tensorflow:loss = 48600.4, step = 99701 (0.870 sec)
INFO:tensorflow:global_step/sec: 118.454
INFO:tensorflow:loss = 26104.3, step = 99801 (0.843 sec)
INFO:tensorflow:global_step/sec: 120.962
INFO:tensorflow:loss = 18790.1, step = 99901 (0.828 sec)
INFO:tensorflow:global_step/sec: 135.306
INFO:tensorflow:loss = 25647.6, step = 100001 (0.735 sec)
INFO:tensorflow:global_step/sec: 156.327
INFO:tensorflow:loss = 26297.2, step = 100101 (0.641 sec)
INFO:tensorflow:global_step/sec: 148.798
INFO:tensorflow:loss = 15346.3, step = 100201 (0.674 sec)
INFO:tensorflow:g

INFO:tensorflow:global_step/sec: 121.217
INFO:tensorflow:loss = 20592.7, step = 107601 (0.825 sec)
INFO:tensorflow:global_step/sec: 129.974
INFO:tensorflow:loss = 967113.0, step = 107701 (0.769 sec)
INFO:tensorflow:global_step/sec: 110.038
INFO:tensorflow:loss = 19771.9, step = 107801 (0.914 sec)
INFO:tensorflow:global_step/sec: 106.267
INFO:tensorflow:loss = 3.01518e+06, step = 107901 (0.941 sec)
INFO:tensorflow:global_step/sec: 102.804
INFO:tensorflow:loss = 263629.0, step = 108001 (0.970 sec)
INFO:tensorflow:global_step/sec: 82.3475
INFO:tensorflow:loss = 408449.0, step = 108101 (1.218 sec)
INFO:tensorflow:global_step/sec: 109.561
INFO:tensorflow:loss = 2.24354e+06, step = 108201 (0.911 sec)
INFO:tensorflow:global_step/sec: 134.359
INFO:tensorflow:loss = 4.21444e+06, step = 108301 (0.742 sec)
INFO:tensorflow:global_step/sec: 126.523
INFO:tensorflow:loss = 115637.0, step = 108401 (0.792 sec)
INFO:tensorflow:global_step/sec: 145.875
INFO:tensorflow:loss = 183511.0, step = 108501 (0.68

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-30-15:50:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_DNNmodel/model.ckpt-109991
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-30-15:50:31
INFO:tensorflow:Saving dict for global step 109991: average_loss = 2192.63, global_step = 109991, loss = 280631.0
RMSE on validation dataset = 46.82553482055664


### Check result on TensorBoard

In [4]:
from google.datalab.ml import TensorBoard
OUTDIR = './trained_DNNmodel'
TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 8527. Click here to access it.

8527